In [12]:
import sys
import os
sys.path.append("../")

In [13]:
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import random

from ngboost.scores import CRPScore, LogScore
from ngboost.learners import default_tree_learner

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error

In [14]:
from uncertainty_estimation.uncertainty_estimation_models import Model, XGBoost, CQR, LightGBM, LSF, NGBoost, TFTPytorchFC, PGBM, LightGBMQuantileRegressor
from uncertainty_estimation.constants import DistEnum, PredEnum

Using C:\Users\daroczi\AppData\Local\torch_extensions\torch_extensions\Cache\py39_cpu as PyTorch extensions root...
No modifications detected for re-loaded extension module split_decision, skipping build step...
Loading extension module split_decision...


FileNotFoundError: Could not find module 'C:\Users\daroczi\AppData\Local\torch_extensions\torch_extensions\Cache\py39_cpu\split_decision\split_decision.pyd' (or one of its dependencies). Try using the full path with constructor syntax.

## 1 Data preparation

In [ ]:
import os

ue_dir_path = os.path.dirname(os.path.dirname(os.getcwd()))
train_path = os.path.join(ue_dir_path, 'datasets', 'bull_train_data.pickle')
test_path = os.path.join(ue_dir_path, 'datasets', 'bull_train_data.pickle')

train_df = pd.read_pickle(train_path)
test_df = pd.read_pickle(test_path)

## REMOVE BEFORE PUBLISHING

In [ ]:
train_df = train_df.iloc[:10000, :]
test_df = test_df.iloc[:1000, :]

In [ ]:
print(len(train_df))
print(len(test_df))

10000
1000


In [ ]:
train_df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,MachineHoursCurrentMeter_na
0,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,TD20,TD20,NaN,NaN,NaN,Medium,"Track Type Tractor, Dozer - 105.0 to 130.0 Hor...",Texas,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Direct Drive,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,3,17,1,17,False,False,False,False,False,False,600998400.0,1
1,1502329,35000.0,1300019,4155,132,99.0,1977,NaN,NaN,1989-01-31,D9H,D9,H,NaN,NaN,Large,"Track Type Tractor, Dozer - 260.0 + Horsepower",Florida,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Semi U,None or Unspecified,NaN,NaN,1989,1,5,31,1,31,True,False,False,False,False,False,602208000.0,1
2,1728883,30000.0,1523610,9105,132,99.0,1986,NaN,NaN,1989-01-31,WA150,WA150,NaN,NaN,NaN,NaN,Wheel Loader - 90.0 to 100.0 Horsepower,Florida,WL,Wheel Loader,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1989,1,5,31,1,31,True,False,False,False,False,False,602208000.0,1
3,1508502,21000.0,1153157,4138,132,99.0,1972,NaN,NaN,1989-01-31,D7F,D7,F,NaN,NaN,Large,"Track Type Tractor, Dozer - 190.0 to 260.0 Hor...",Florida,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Angle,None or Unspecified,NaN,NaN,1989,1,5,31,1,31,True,False,False,False,False,False,602208000.0,1
4,1452578,33000.0,1544443,3854,132,99.0,1974,NaN,NaN,1989-01-31,966C,966,C,NaN,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,Florida,WL,Wheel Loader,NaN,EROPS,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,1989,1,5,31,1,31,True,False,False,False,False,False,602208000.0,1


In [ ]:
test_df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,MachineHoursCurrentMeter_na
21055,1212349,30000.0,1003126,13247,121,3.0,2005,3013.0,Medium,2011-02-25,580MII,580,M,II,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Texas,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Extended,Standard,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,2,8,25,4,56,False,False,False,False,False,False,1.298592e+09,0
20555,1210735,15500.0,791516,26095,121,3.0,2008,1251.0,Medium,2011-02-25,PT50,PT50,NaN,NaN,NaN,NaN,Skid Steer Loader - 1751.0 to 2201.0 Lb Operat...,Florida,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,Manual,Yes,Yes,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,2,8,25,4,56,False,False,False,False,False,False,1.298592e+09,0
20948,1212005,12000.0,1068292,17311,121,3.0,2004,1791.0,Medium,2011-02-25,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,Georgia,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,Manual,Yes,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,2,8,25,4,56,False,False,False,False,False,False,1.298592e+09,0
20819,1211658,12000.0,109383,3537,121,3.0,1000,13411.0,High,2011-02-25,416B,416,B,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,North Carolina,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,Reversible,No,Standard,Powershuttle,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,2,8,25,4,56,False,False,False,False,False,False,1.298592e+09,0
21267,1212872,42000.0,1056052,18139,121,3.0,2004,2515.0,Low,2011-02-25,EC160BLC,EC160,B,NaN,LC,Small,"Hydraulic Excavator, Track - 16.0 to 19.0 Metr...",Michigan,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,"9' 10""",None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2011,2,8,25,4,56,False,False,False,False,False,False,1.298592e+09,0


After the data is loaded the train data is sorted after saledate.

In [ ]:
train_df = train_df.sort_values(by='saledate', ascending=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
cat_vars = ['SalesID', 'YearMade', 'MachineID', 'ModelID', 'datasource', 'auctioneerID', 'UsageBand', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 
            'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension', 
            'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow', 
            'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls', 'Differential_Type', 'Steering_Controls', 
            'saleYear', 'saleMonth', 'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear', 'saleIs_month_end', 'saleIs_month_start', 'saleIs_quarter_end', 'saleIs_quarter_start', 'saleIs_year_end', 
            'saleIs_year_start', 'MachineHoursCurrentMeter_na'
           ]

cont_vars = ['MachineHoursCurrentMeter', 'saleElapsed']

for col in cat_vars:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

<command-2031129775214582>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[col] = test_df[col].astype('category')


## 2 Vectorizer

The vectorizer takes tabular data and transforms them into a better format for use in machine learning models. A vectorizer should provide two methods, a fit_transform() and a transform() method. The first is used to train the vectorizer. transform() is used just to apply the trained vectorizer.
The data in this example contains categorical and continuous features. Categorial features are selected by CategoricalSelector while continuous features are selected by ItemSelector. Through FeatureUnion it is possible to select features and then do missing value handeling or transform a feature.
For an example vectorizer the features ProductSize, ModelID, state, ProductGroup and YearMade are chosen. Missing value handling is done for every feature except YearMade.

In [ ]:
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline

import feature_engine.imputation as fe
import category_encoders as ce

class CategoricalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field

    def fit(self, x, y=None):
        return self

    def transform(self, dataframe):
        dt = dataframe[self.field].dtype
        return dataframe[[self.field]]


class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field

    def fit(self, x, y=None):
        return self

    def transform(self, dataframe):
        dt = dataframe[self.field].dtype
        if is_categorical_dtype(dt):
            return dataframe[self.field].cat.codes[:, None]
        elif is_numeric_dtype(dt):
            return dataframe[self.field][:, None]
        else:
            return dataframe[self.field]

def create_feature_vectorizer_without_nan():
    vectorizer_tree = FeatureUnion([
        ('SalesID',
         Pipeline([('select', CategoricalSelector('SalesID')),
                   ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('YearMade',
         Pipeline([('select', CategoricalSelector('YearMade')),
                   ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('MachineID',
         Pipeline([
             ('select', CategoricalSelector('MachineID')),
             ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('ModelID',
         Pipeline(
             [('select', CategoricalSelector('ModelID')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('datasource',
         Pipeline(
             [('select', CategoricalSelector('datasource')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('auctioneerID',
         Pipeline(
             [('select', CategoricalSelector('auctioneerID')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('UsageBand',
         Pipeline(
             [('select', CategoricalSelector('UsageBand')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiModelDesc',
         Pipeline(
             [('select', CategoricalSelector('fiModelDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiBaseModel',
         Pipeline(
             [('select', CategoricalSelector('fiBaseModel')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiSecondaryDesc',
         Pipeline(
             [('select', CategoricalSelector('fiSecondaryDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiModelSeries',
         Pipeline(
             [('select', CategoricalSelector('fiModelSeries')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiModelDescriptor',
         Pipeline(
             [('select', CategoricalSelector('fiModelDescriptor')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('ProductSize',
         Pipeline(
             [('select', CategoricalSelector('ProductSize')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('fiProductClassDesc',
         Pipeline(
             [('select', CategoricalSelector('fiProductClassDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('state',
         Pipeline(
             [('select', CategoricalSelector('state')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('ProductGroup',
         Pipeline(
             [('select', CategoricalSelector('ProductGroup')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('ProductGroupDesc',
         Pipeline(
             [('select', CategoricalSelector('ProductGroupDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Drive_System',
         Pipeline(
             [('select', CategoricalSelector('Drive_System')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Enclosure',
         Pipeline(
             [('select', CategoricalSelector('Enclosure')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Forks',
         Pipeline(
             [('select', CategoricalSelector('Forks')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Pad_Type',
         Pipeline(
             [('select', CategoricalSelector('Pad_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Ride_Control',
         Pipeline(
             [('select', CategoricalSelector('Ride_Control')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Stick',
         Pipeline(
             [('select', CategoricalSelector('Stick')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Transmission',
         Pipeline(
             [('select', CategoricalSelector('Transmission')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Turbocharged',
         Pipeline(
             [('select', CategoricalSelector('Turbocharged')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Blade_Extension',
         Pipeline(
             [('select', CategoricalSelector('Blade_Extension')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Blade_Width',
         Pipeline(
             [('select', CategoricalSelector('Blade_Width')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Enclosure_Type',
         Pipeline(
             [('select', CategoricalSelector('Enclosure_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Engine_Horsepower',
         Pipeline(
             [('select', CategoricalSelector('Engine_Horsepower')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Hydraulics',
         Pipeline(
             [('select', CategoricalSelector('Hydraulics')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Pushblock',
         Pipeline(
             [('select', CategoricalSelector('Pushblock')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Ripper',
         Pipeline(
             [('select', CategoricalSelector('Ripper')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Scarifier',
         Pipeline(
             [('select', CategoricalSelector('Scarifier')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Tip_Control',
         Pipeline(
             [('select', CategoricalSelector('Tip_Control')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Tire_Size',
         Pipeline(
             [('select', CategoricalSelector('Tire_Size')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Coupler',
         Pipeline(
             [('select', CategoricalSelector('Coupler')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Coupler_System',
         Pipeline(
             [('select', CategoricalSelector('Coupler_System')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Grouser_Tracks',
         Pipeline(
             [('select', CategoricalSelector('Grouser_Tracks')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Hydraulics_Flow',
         Pipeline(
             [('select', CategoricalSelector('Hydraulics_Flow')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Track_Type',
         Pipeline(
             [('select', CategoricalSelector('Track_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Undercarriage_Pad_Width',
         Pipeline(
             [('select', CategoricalSelector('Undercarriage_Pad_Width')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Stick_Length',
         Pipeline(
             [('select', CategoricalSelector('Stick_Length')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Thumb',
         Pipeline(
             [('select', CategoricalSelector('Thumb')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Pattern_Changer',
         Pipeline(
             [('select', CategoricalSelector('Pattern_Changer')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Grouser_Type',
         Pipeline(
             [('select', CategoricalSelector('Grouser_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Backhoe_Mounting',
         Pipeline(
             [('select', CategoricalSelector('Backhoe_Mounting')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Blade_Type',
         Pipeline(
             [('select', CategoricalSelector('Blade_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Travel_Controls',
         Pipeline(
             [('select', CategoricalSelector('Travel_Controls')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Differential_Type',
         Pipeline(
             [('select', CategoricalSelector('Differential_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('Steering_Controls',
         Pipeline(
             [('select', CategoricalSelector('Steering_Controls')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleYear',
         Pipeline(
             [('select', CategoricalSelector('saleYear')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleMonth',
         Pipeline(
             [('select', CategoricalSelector('saleMonth')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleWeek',
         Pipeline(
             [('select', CategoricalSelector('saleWeek')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleDay',
         Pipeline(
             [('select', CategoricalSelector('saleDay')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleDayofweek',
         Pipeline(
             [('select', CategoricalSelector('saleDayofweek')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleDayofyear',
         Pipeline(
             [('select', CategoricalSelector('saleDayofyear')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_month_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_month_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_month_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_month_start')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_quarter_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_quarter_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_year_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('saleIs_year_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('MachineHoursCurrentMeter_na',
         Pipeline(
             [('select', CategoricalSelector('MachineHoursCurrentMeter_na')),
              ('oe', ce.OrdinalEncoder(handle_missing="value", handle_unknown='value'))])),
        ('MachineHoursCurrentMeter',Pipeline([('select', ItemSelector('MachineHoursCurrentMeter')),
                                             ('fe', fe.ArbitraryNumberImputer(arbitrary_number=-1))])),
        ('saleElapsed',Pipeline([('select', ItemSelector('saleElapsed'))])),
    ], n_jobs=1)

    return vectorizer_tree

def create_feature_vectorizer_with_nan():
    vectorizer_tree = FeatureUnion([
        ('SalesID',
         Pipeline([('select', CategoricalSelector('SalesID')),
                   ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('YearMade',
         Pipeline([('select', CategoricalSelector('YearMade')),
                   ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('MachineID',
         Pipeline([
             ('select', CategoricalSelector('MachineID')),
             ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('ModelID',
         Pipeline(
             [('select', CategoricalSelector('ModelID')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('datasource',
         Pipeline(
             [('select', CategoricalSelector('datasource')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('auctioneerID',
         Pipeline(
             [('select', CategoricalSelector('auctioneerID')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('UsageBand',
         Pipeline(
             [('select', CategoricalSelector('UsageBand')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiModelDesc',
         Pipeline(
             [('select', CategoricalSelector('fiModelDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiBaseModel',
         Pipeline(
             [('select', CategoricalSelector('fiBaseModel')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiSecondaryDesc',
         Pipeline(
             [('select', CategoricalSelector('fiSecondaryDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiModelSeries',
         Pipeline(
             [('select', CategoricalSelector('fiModelSeries')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiModelDescriptor',
         Pipeline(
             [('select', CategoricalSelector('fiModelDescriptor')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('ProductSize',
         Pipeline(
             [('select', CategoricalSelector('ProductSize')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('fiProductClassDesc',
         Pipeline(
             [('select', CategoricalSelector('fiProductClassDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('state',
         Pipeline(
             [('select', CategoricalSelector('state')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('ProductGroup',
         Pipeline(
             [('select', CategoricalSelector('ProductGroup')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('ProductGroupDesc',
         Pipeline(
             [('select', CategoricalSelector('ProductGroupDesc')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Drive_System',
         Pipeline(
             [('select', CategoricalSelector('Drive_System')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Enclosure',
         Pipeline(
             [('select', CategoricalSelector('Enclosure')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Forks',
         Pipeline(
             [('select', CategoricalSelector('Forks')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Pad_Type',
         Pipeline(
             [('select', CategoricalSelector('Pad_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Ride_Control',
         Pipeline(
             [('select', CategoricalSelector('Ride_Control')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Stick',
         Pipeline(
             [('select', CategoricalSelector('Stick')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Transmission',
         Pipeline(
             [('select', CategoricalSelector('Transmission')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Turbocharged',
         Pipeline(
             [('select', CategoricalSelector('Turbocharged')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Blade_Extension',
         Pipeline(
             [('select', CategoricalSelector('Blade_Extension')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Blade_Width',
         Pipeline(
             [('select', CategoricalSelector('Blade_Width')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Enclosure_Type',
         Pipeline(
             [('select', CategoricalSelector('Enclosure_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Engine_Horsepower',
         Pipeline(
             [('select', CategoricalSelector('Engine_Horsepower')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Hydraulics',
         Pipeline(
             [('select', CategoricalSelector('Hydraulics')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Pushblock',
         Pipeline(
             [('select', CategoricalSelector('Pushblock')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Ripper',
         Pipeline(
             [('select', CategoricalSelector('Ripper')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Scarifier',
         Pipeline(
             [('select', CategoricalSelector('Scarifier')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Tip_Control',
         Pipeline(
             [('select', CategoricalSelector('Tip_Control')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Tire_Size',
         Pipeline(
             [('select', CategoricalSelector('Tire_Size')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Coupler',
         Pipeline(
             [('select', CategoricalSelector('Coupler')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Coupler_System',
         Pipeline(
             [('select', CategoricalSelector('Coupler_System')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Grouser_Tracks',
         Pipeline(
             [('select', CategoricalSelector('Grouser_Tracks')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Hydraulics_Flow',
         Pipeline(
             [('select', CategoricalSelector('Hydraulics_Flow')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Track_Type',
         Pipeline(
             [('select', CategoricalSelector('Track_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Undercarriage_Pad_Width',
         Pipeline(
             [('select', CategoricalSelector('Undercarriage_Pad_Width')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Stick_Length',
         Pipeline(
             [('select', CategoricalSelector('Stick_Length')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Thumb',
         Pipeline(
             [('select', CategoricalSelector('Thumb')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Pattern_Changer',
         Pipeline(
             [('select', CategoricalSelector('Pattern_Changer')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Grouser_Type',
         Pipeline(
             [('select', CategoricalSelector('Grouser_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Backhoe_Mounting',
         Pipeline(
             [('select', CategoricalSelector('Backhoe_Mounting')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Blade_Type',
         Pipeline(
             [('select', CategoricalSelector('Blade_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Travel_Controls',
         Pipeline(
             [('select', CategoricalSelector('Travel_Controls')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Differential_Type',
         Pipeline(
             [('select', CategoricalSelector('Differential_Type')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('Steering_Controls',
         Pipeline(
             [('select', CategoricalSelector('Steering_Controls')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleYear',
         Pipeline(
             [('select', CategoricalSelector('saleYear')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleMonth',
         Pipeline(
             [('select', CategoricalSelector('saleMonth')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleWeek',
         Pipeline(
             [('select', CategoricalSelector('saleWeek')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleDay',
         Pipeline(
             [('select', CategoricalSelector('saleDay')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleDayofweek',
         Pipeline(
             [('select', CategoricalSelector('saleDayofweek')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleDayofyear',
         Pipeline(
             [('select', CategoricalSelector('saleDayofyear')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_month_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_month_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_month_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_month_start')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_quarter_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_quarter_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_year_end',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        ('saleIs_year_start',
         Pipeline(
             [('select', CategoricalSelector('saleIs_quarter_end')),
              ('oe', ce.OrdinalEncoder(handle_missing="return_nan", handle_unknown='return_nan'))])),
        
        ('MachineHoursCurrentMeter',Pipeline([('select', ItemSelector('MachineHoursCurrentMeter'))])),
        ('saleElapsed',Pipeline([('select', ItemSelector('saleElapsed'))])),   
    ], n_jobs=1)

    return vectorizer_tree

In [ ]:
vectorizer_without_nan = create_feature_vectorizer_without_nan()
vectorizer_with_nan = create_feature_vectorizer_with_nan()

In [ ]:
print(vectorizer_without_nan.fit_transform(train_df).shape)
print(vectorizer_with_nan.fit_transform(train_df).shape)

(10000, 65)
(10000, 64)


## 3 Model Application

In [ ]:
TARGET = 'SalePrice'
target_transformer = "log"

10% for the test dataset and then a 80/20 train-val split on the remaining 90% of the data.

In [ ]:
train_val_df, valid_df = model_selection.train_test_split(train_df, test_size=0.2, random_state=1, shuffle=False)
print(len(train_val_df), len(valid_df))

8000 2000


## 3.0 LightGBM

In [ ]:
lightgbm_params = {
    "boosting_type": 'gbdt',
    "objective": 'regression',
    "n_jobs": -1, 
    "min_split_gain": 0.0,
    "min_data_in_leaf": 20,
    "max_bin": 255,
    "num_leaves": 31,
    "lambda": 0.0,
    "max_depth": -1,
    "learning_rate": 0.1,
    "feature_fraction": 1.0,
    "bagging_fraction": 1.0,
    "bagging_freq": 0,
    "seed": 1,
    "n_estimators": 2000,
}

early_stopping_round = 20

start_time = time.perf_counter()

# fitting model on train set with early stopping on valid set
lightgbm_reg = LightGBM(vectorizer_with_nan, target_transformer=target_transformer)
lightgbm_fit_params = {"early_stopping_round": early_stopping_round}
lightgbm_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=valid_df[TARGET], params=lightgbm_fit_params, verbose=True)
lightgbm_best_iteration = lightgbm_reg.best_iteration
print("Early stopping performed. Best iteration:", lightgbm_best_iteration)
    
# fitting model on train+val set with best_iteration
lightgbm_full_train_reg = LightGBM(vectorizer_with_nan, target_transformer=target_transformer)
start_time = time.perf_counter()
lightgbm_full_train_params = {**lightgbm_params, "n_estimators": lightgbm_best_iteration}
lightgbm_full_train_reg.fit(train_df, TARGET, params=lightgbm_full_train_params, verbose=True)
    
# predicting on test set with our fully trained model
lightgbm_pred = lightgbm_full_train_reg.predict(test_df)
lightgbm_metrics = lightgbm_full_train_reg.metrics(test_df[TARGET], lightgbm_pred)
lightgbm_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], lightgbm_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
lightgbm_metrics['time'] = full_time

[1]	valid_0's l2: 0.345786
[2]	valid_0's l2: 0.309208
[3]	valid_0's l2: 0.280003
[4]	valid_0's l2: 0.254153
[5]	valid_0's l2: 0.232837
[6]	valid_0's l2: 0.216484
[7]	valid_0's l2: 0.200928
[8]	valid_0's l2: 0.190094
[9]	valid_0's l2: 0.178772
[10]	valid_0's l2: 0.170539
[11]	valid_0's l2: 0.160791
[12]	valid_0's l2: 0.153285
[13]	valid_0's l2: 0.146633
[14]	valid_0's l2: 0.141238
[15]	valid_0's l2: 0.136825
[16]	valid_0's l2: 0.133106
[17]	valid_0's l2: 0.130679
[18]	valid_0's l2: 0.12664
[19]	valid_0's l2: 0.123658
[20]	valid_0's l2: 0.121901
[21]	valid_0's l2: 0.119358
[22]	valid_0's l2: 0.116643
[23]	valid_0's l2: 0.115199
[24]	valid_0's l2: 0.114278
[25]	valid_0's l2: 0.113212
[26]	valid_0's l2: 0.11176
[27]	valid_0's l2: 0.109494
[28]	valid_0's l2: 0.109059
[29]	valid_0's l2: 0.108639
[30]	valid_0's l2: 0.108337
[31]	valid_0's l2: 0.107379
[32]	valid_0's l2: 0.105916
[33]	valid_0's l2: 0.10479
[34]	valid_0's l2: 0.104034
[35]	valid_0's l2: 0.104047
[36]	valid_0's l2: 0.103427
[37]

In [ ]:
lightgbm_metrics

Out[37]: {'mse': 678953872.6248896,
 'mae': 18055.390726528203,
 'rmse': 26056.743323464074,
 'mape': 0.5647115457207417,
 'rmspe': 0.7869228072859047,
 'rmsle': 0.7722770016648014,
 'time': 19.3}

## 3.1 Bootstrapping (Data sampling)

In [ ]:
# fitting model with parameters obtained from early stopping with train_val set
n_resamples = 100
case_resampling_results = np.zeros(shape = (len(test_df), n_resamples))

start_time = time.perf_counter()

train_temp = train_df.reset_index(drop=True).copy()
for i in range(n_resamples):
    print(f"resampling i: {i+1}")

    if i == 0:
        train_resampled = train_temp
    else:
        train_resampled = train_temp.iloc[random.choices(list(range(len(train_temp))), k = len(train_temp))].reset_index(drop=True)

    lightgbm_full_train_params = {**lightgbm_params, "n_estimators": lightgbm_best_iteration}
    lgb_model = LightGBM(vectorizer_with_nan, target_transformer=target_transformer)
    lgb_model.fit(train_resampled, TARGET, params=lightgbm_full_train_params, verbose=True)

    # predicting on test set with our fully trained model
    bootstrap_pred = lgb_model.predict(test_df)
    case_resampling_results[:, i] = bootstrap_pred[PredEnum.POINT_ESTIMATES]

bootstrap_samples = case_resampling_results
bootstrap_quantiles = np.concatenate((np.quantile(case_resampling_results, q = 0.1, axis = 1)[:,np.newaxis], np.quantile(case_resampling_results, q = 0.9, axis = 1)[:,np.newaxis]), axis = 1)
bootstrap_point_pred = case_resampling_results.mean(axis = 1)

bootstrap_metrics = {}
bootstrap_metrics['rmse'] = Model.rmse(test_df[TARGET], bootstrap_point_pred)
bootstrap_metrics['rmspe'] = Model.rmspe(test_df[TARGET], bootstrap_point_pred)
bootstrap_metrics['rmsle'] = np.sqrt((( np.log1p(test_df[TARGET]) - np.log1p(bootstrap_point_pred))**2).mean())
bootstrap_metrics['avg_interval_length'] = Model.avg_interval_length(bootstrap_quantiles)
bootstrap_metrics['sharpness'] = Model.avg_interval_length(bootstrap_quantiles)
bootstrap_metrics['coverage'] = Model.coverage(test_df[TARGET], bootstrap_quantiles)
bootstrap_metrics['crps'] = Model.crps(test_df[TARGET], case_resampling_results)
bootstrap_metrics['nll_from_samples'] = Model.neg_log_likelihood_with_kde(np.array(test_df[TARGET]), case_resampling_results)

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
bootstrap_metrics['time'] = full_time

resampling i: 1
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.0. Current value: lambda_l2=0.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
Elapsed time for fitting LightGBM model: 15.26 s
resampling i: 2
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.0. Current value: lambda_l2=0.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored.

In [ ]:
bootstrap_metrics

Out[35]: {'rmse': 20608.80317881689,
 'rmspe': 0.8446050038058508,
 'rmsle': 0.5816251066371448,
 'avg_interval_length': 23793.139689934058,
 'sharpness': 23793.139689934058,
 'coverage': 0.449,
 'crps': 11252.375014636282,
 'nll_from_samples': 14.454147498139696,
 'time': 1930.17}

## 3.2 Quantile Regression

In [ ]:
lightgbm_quant_params = {
    "boosting_type": 'gbdt',
    "objective": 'quantile',
    "n_jobs": -1, 
    "min_split_gain": 0.0,
    "min_data_in_leaf": 20,
    "max_bin": 255,
    "num_leaves": 31,
    "lambda": 0.0,
    "max_depth": -1,
    "learning_rate": 0.1,
    "feature_fraction": 1.0,
    "bagging_fraction": 1.0,
    "bagging_freq": 0,
    "seed": 1,
    "n_estimators": 2000, 
}

early_stopping_round = 20

quantiles = [0.1, 0.5, 0.9]

start_time = time.perf_counter()  

# fitting model on train set with early stopping on valid set
lightgbm_quant_reg = LightGBMQuantileRegressor(vectorizer_with_nan, target_transformer=target_transformer)
lightgbm_quant_fit_params = {**lightgbm_quant_params, "early_stopping_round": early_stopping_round}
lightgbm_quant_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=valid_df[TARGET], params=lightgbm_quant_fit_params, quantiles=quantiles, verbose=True)
lightgbm_quant_best_iteration = int(np.mean(list(lightgbm_quant_reg.best_iterations.values())))
print("Early stopping performed. Best iteration:", lightgbm_quant_best_iteration)

    # fitting model on train+val set with best_iteration
lightgbm_full_train_quant_reg = LightGBMQuantileRegressor(vectorizer_with_nan, target_transformer=target_transformer)
lightgbm_quant_full_train_params = {**lightgbm_quant_params, "n_estimators": lightgbm_quant_best_iteration}
lightgbm_full_train_quant_reg.fit(train_df, TARGET, params=lightgbm_quant_full_train_params, quantiles=quantiles, verbose=True)
lightgbm_full_train_quant_reg.best_iterations = lightgbm_quant_reg.best_iterations
    
# predicting on test set with our fully trained model
lightgbm_quant_pred = lightgbm_full_train_quant_reg.predict(test_df)
lightgbm_quant_metrics = lightgbm_full_train_quant_reg.metrics(test_df[TARGET], lightgbm_quant_pred, confidence_interval_quantiles = [0.1, 0.9])
lightgbm_quant_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], lightgbm_quant_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
lightgbm_quant_metrics['time'] = full_time

[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.0. Current value: lambda_l2=0.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[1]	valid_0's quantile: 0.0904261
[2]	valid_0's quantile: 0.0883992
[3]	valid_0's quantile: 0.087258
[4]	valid_0's quantile: 0.085985
[5]	valid_0's quantile: 0.0849389
[6]	valid_0's quantile: 0.0841454
[7]	valid_0's quantile: 0.083187
[8]	valid_0's quantile: 0.0819954
[9]	valid_0's quantile: 0.0813039
[10]	valid_0's quantile: 0.0799666
[11]	valid_0's quantile: 0.0791673
[12]	valid_0's quantile: 0.0784959

In [ ]:
lightgbm_quant_metrics

Out[39]: {'mse': 522981241.3881084,
 'mae': 15804.918052802448,
 'rmse': 22868.783119967455,
 'mape': 0.5421740504588072,
 'rmspe': 0.7993666273487413,
 'avg_interval_length': 20568.942195673008,
 'sharpness': 20568.942195673008,
 'coverage': 0.454,
 'rmsle': 0.6428176834390774,
 'time': 55.02}

## 3.3 NGBoost

### 3.3.1 NGBoost with NLL

In [ ]:
from ngboost.scores import LogScore, CRPScore
from lightgbm import LGBMRegressor

ngboost_nll_base_params = {
    'boosting': 'rf',
    'n_estimators': 1,
    'bagging_fraction': 0.99, 
    'bagging_freq': 1 
}

learner = LGBMRegressor(**ngboost_nll_base_params)

ngboost_nll_params = {
    'Score':LogScore,
    'Base':learner, 
    'natural_gradient':True,
    "learning_rate": 0.1, 
    "n_estimators": 2000, 
    "col_sample": 1.0, 
    "minibatch_frac": 1.0,
    "random_state": 1
}

early_stopping_round = 20
quantiles = [0.05, 0.1, 0.5, 0.9, 0.95]

start_time = time.perf_counter()

# fitting model on train set with early stopping on valid set
ngboost_nll_early_stopping_params = {**ngboost_nll_params, "early_stopping_rounds": early_stopping_round}
ngboost_nll_reg = NGBoost(vectorizer_without_nan, target_transformer=target_transformer, distribution=DistEnum.NORMAL, **ngboost_nll_early_stopping_params)
ngboost_nll_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=np.array(valid_df[TARGET]), verbose=True)
ngboost_best_iteration = ngboost_nll_reg.best_iteration
print("Early stopping performed. Best iteration:", ngboost_best_iteration)

# fitting model on train+val set with best_iteration
ngboost_full_train_params = {**ngboost_nll_params, "n_estimators": ngboost_best_iteration}
ngboost_nll_full_train_reg = NGBoost(vectorizer_without_nan, target_transformer=target_transformer, distribution=DistEnum.NORMAL, **ngboost_full_train_params)
ngboost_nll_full_train_reg.fit(train_df, TARGET, verbose=True)

# predicting on test set with our fully trained model
ngboost_nll_pred = ngboost_nll_full_train_reg.predict(test_df, quantiles=quantiles, prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES, PredEnum.SAMPLES, PredEnum.DISTRIBUTION_PARAMS], sample_size=400)

ngboost_nll_metrics = ngboost_nll_full_train_reg.metrics(np.array(test_df[TARGET]), ngboost_nll_pred, confidence_interval_quantiles=[0.1,0.9])
ngboost_nll_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], ngboost_nll_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
ngboost_nll_metrics['time'] = full_time

[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0 will be ignored. Current value: bagging_fraction=0.99
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0 will be ignored. Current value: bagging_fraction=0.99
[iter 0] loss=0.9957 val_loss=0.8741 scale=1.0000 norm=0.7575
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0

In [ ]:
ngboost_nll_metrics

Out[41]: {'mse': 605937302.5406529,
 'mae': 16828.947411224428,
 'rmse': 24615.793762148984,
 'mape': 0.5400496130445711,
 'rmspe': 0.788618064163609,
 'avg_interval_length': 12612.001645058974,
 'sharpness': 12612.001645058974,
 'coverage': 0.297,
 'crps': 14532.626391527523,
 'nll_from_samples': 52.29109683710777,
 'nll': 15.340840835736444,
 'rmsle': 0.7151712848629788,
 'time': 1453.15}

### 3.3.2 NGBoost with CRPS

In [ ]:
from ngboost.scores import LogScore, CRPScore
from lightgbm import LGBMRegressor

ngboost_crps_base_params = {
    'boosting': 'rf',
    'n_estimators': 1,
    'bagging_fraction': 0.99, 
    'bagging_freq': 1 
}

learner = LGBMRegressor(**ngboost_crps_base_params)

ngboost_crps_params = {
    'Score':CRPScore,
    'Base':learner,
    'natural_gradient':True,
    "learning_rate": 0.1, 
    "n_estimators": 2000,
    "col_sample": 1.0, 
    "minibatch_frac": 1.0, 
    "random_state": 1
}
    
early_stopping_round = 20
quantiles = [0.05, 0.1, 0.5, 0.9, 0.95]

start_time = time.perf_counter()


# fitting model on train set with early stopping on valid set
ngboost_crps_early_stopping_params = {**ngboost_crps_params, "early_stopping_rounds": early_stopping_round}
ngboost_crps_reg = NGBoost(vectorizer_without_nan, target_transformer=target_transformer, distribution=DistEnum.NORMAL, **ngboost_crps_early_stopping_params)
ngboost_crps_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=np.array(valid_df[TARGET]), verbose=True)
best_iteration = ngboost_crps_reg.best_iteration
print("Early stopping performed. Best iteration:", best_iteration)

# fitting model on train+val set with best_iteration
ngboost_full_train_params = {**ngboost_crps_params, "n_estimators": best_iteration}
ngboost_crps_full_train_reg = NGBoost(vectorizer_without_nan, target_transformer=target_transformer, distribution=DistEnum.NORMAL, **ngboost_full_train_params)
ngboost_crps_full_train_reg.fit(train_df, TARGET, verbose=True)
    
# predicting on test set with our fully trained model
ngboost_crps_pred = ngboost_crps_full_train_reg.predict(test_df, quantiles=quantiles, prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES, PredEnum.SAMPLES, PredEnum.DISTRIBUTION_PARAMS], sample_size=400)

ngboost_crps_metrics = ngboost_crps_full_train_reg.metrics(np.array(test_df[TARGET]), ngboost_crps_pred, confidence_interval_quantiles=[0.1,0.9])
ngboost_crps_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], ngboost_crps_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
ngboost_crps_metrics['time'] = full_time

[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0 will be ignored. Current value: bagging_fraction=0.99
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0 will be ignored. Current value: bagging_fraction=0.99
[iter 0] loss=0.3755 val_loss=0.3163 scale=1.0000 norm=1.4716
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0

In [ ]:
ngboost_crps_metrics

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2031129775214605> in <cell line: 1>()
----> 1 ngboost_crps_metrics

NameError: name 'ngboost_crps_metrics' is not defined

## 3.4 PGBM

### 3.4.1 PGBM normal

In [ ]:
pgbm_normal_params = {
    'derivatives': 'exact',
    'distribution': 'normal',
    'device': 'gpu',
    'gpu_device_id': 0,
    "n_jobs": -1,
    "min_split_gain": 0.0,
    "min_data_in_leaf": 20,
    "max_bin": 255,
    "max_leaves": 31,
    "max_depth": -1,
    "learning_rate": 0.1,
    "n_estimators": 2000,
    "feature_fraction": 1.0,
    "bagging_fraction": 1.0,
    "seed": 1,
    "lambda": 0.0
}

early_stopping_round = 20
quantiles = [0.05, 0.1, 0.5, 0.9, 0.95]

start_time = time.perf_counter()

# fitting model on train set with early stopping on valid set
pgbm_fit_params = {**pgbm_normal_params, "early_stopping_round": early_stopping_round}
pgbm_normal_reg = PGBM(vectorizer_without_nan, target_transformer=target_transformer)
pgbm_normal_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=np.array(valid_df[TARGET]), params=pgbm_fit_params, apply_optimize_distribution=False, verbose=True)
pgbm_best_iteration = pgbm_normal_reg.best_iteration
print("Early stopping performed. Best iteration:", pgbm_best_iteration)
    
# fitting model on train+val set with best_iteration
pgbm_normal_full_fit_params = {**pgbm_normal_params, "n_estimators": pgbm_best_iteration}
pgbm_normal_full_reg = PGBM(vectorizer_without_nan, target_transformer=target_transformer)
pgbm_normal_full_reg.fit(train_df, TARGET, params=pgbm_normal_full_fit_params, apply_optimize_distribution=False, verbose=True)
    
# predicting on test set with our fully trained model
pgbm_normal_pred = pgbm_normal_full_reg.predict(test_df, quantiles=quantiles, prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES, PredEnum.SAMPLES, PredEnum.DISTRIBUTION_PARAMS], sample_size=300)

pgbm_normal_metrics = pgbm_normal_full_reg.metrics(np.array(test_df[TARGET]), pgbm_normal_pred, confidence_interval_quantiles=[0.1,0.9])
pgbm_normal_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], pgbm_normal_pred[PredEnum.POINT_ESTIMATES]))
    
end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
pgbm_normal_metrics['time'] = full_time

Training on GPU
Estimator 0/2000, Train metric: 0.6161, Validation metric: 0.5917
Estimator 1/2000, Train metric: 0.5852, Validation metric: 0.5623
Estimator 2/2000, Train metric: 0.5552, Validation metric: 0.5354
Estimator 3/2000, Train metric: 0.5278, Validation metric: 0.5106
Estimator 4/2000, Train metric: 0.5072, Validation metric: 0.4928
Estimator 5/2000, Train metric: 0.4879, Validation metric: 0.4745
Estimator 6/2000, Train metric: 0.4697, Validation metric: 0.4584
Estimator 7/2000, Train metric: 0.4526, Validation metric: 0.4427
Estimator 8/2000, Train metric: 0.4397, Validation metric: 0.4315
Estimator 9/2000, Train metric: 0.4299, Validation metric: 0.4243
Estimator 10/2000, Train metric: 0.4193, Validation metric: 0.4160
Estimator 11/2000, Train metric: 0.4114, Validation metric: 0.4086
Estimator 12/2000, Train metric: 0.4039, Validation metric: 0.4039
Estimator 13/2000, Train metric: 0.3933, Validation metric: 0.3959
Estimator 14/2000, Train metric: 0.3848, Validation metr

In [ ]:
pgbm_normal_metrics

### 3.4.2 PGBM best dist.

In [ ]:
pgbm_best_dist_params = {
    'derivatives': 'exact',
    'distribution': 'normal',
    'device': 'gpu',
    'gpu_device_id': 0,
    "n_jobs": -1, 
    "min_split_gain": 0.0,
    "min_data_in_leaf": 20,
    "max_bin": 255,
    "max_leaves": 31,
    "max_depth": -1,
    "learning_rate": 0.1,
    "n_estimators": 2000,
    "feature_fraction": 1.0,
    "bagging_fraction": 1.0,
    "seed": 1,
    "lambda": 0.0
}

early_stopping_round = 20 
quantiles = [0.05, 0.1, 0.5, 0.9, 0.95]

start_time = time.perf_counter()
    
# fitting model on train set with early stopping on valid set
fit_params = {**pgbm_best_dist_params, "early_stopping_round": early_stopping_round}
pgbm_best_dist_reg = PGBM(vectorizer_without_nan, target_transformer=target_transformer)
pgbm_best_dist_reg.fit(train_val_df, TARGET, X_val=valid_df, y_val=np.array(valid_df[TARGET]), params=fit_params, apply_optimize_distribution=True, verbose=True)
best_distribution = pgbm_best_dist_reg.model.distribution
best_tree_correlation = pgbm_best_dist_reg.model.tree_correlation
best_iteration = pgbm_best_dist_reg.best_iteration
print("Early stopping performed. Best iteration:", best_iteration)
print("Best distribution found:", pgbm_best_iteration)
print("Best tree correlation found:", best_tree_correlation)
    
# fitting model on train+val set with best_iteration
pgbm_best_dist_full_fit_params = {**pgbm_best_dist_params, "n_estimators": best_iteration}
pgbm_best_dist_full_reg = PGBM(vectorizer_without_nan, target_transformer=target_transformer)
pgbm_best_dist_full_reg.fit(train_df, TARGET, params=pgbm_best_dist_full_fit_params, apply_optimize_distribution=False, verbose=True)
 
# predicting on test set with our fully trained model
pgbm_best_dist_full_reg.model.distribution = best_distribution
pgbm_best_dist_full_reg.model.tree_correlation = best_tree_correlation
pgbm_best_dist_pred = pgbm_best_dist_full_reg.predict(test_df, quantiles=quantiles, prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES, PredEnum.SAMPLES, PredEnum.DISTRIBUTION_PARAMS], sample_size=300)

pgbm_best_dist_metrics = pgbm_best_dist_full_reg.metrics(np.array(test_df[TARGET]), pgbm_best_dist_pred, confidence_interval_quantiles=[0.1,0.9])
pgbm_best_dist_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], pgbm_best_dist_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
pgbm_best_dist_metrics['time'] = full_time

In [ ]:
pgbm_best_dist_metrics

Out[43]: {'mse': 206206368.51389858,
 'mae': 10311.899469984723,
 'rmse': 14359.88748263365,
 'mape': 0.3917285184309194,
 'rmspe': 0.5554253763030468,
 'avg_interval_length': 38064.5494608078,
 'sharpness': 38064.5494608078,
 'coverage': 0.7683547370971651,
 'crps': 7458.391948836988,
 'nll_from_samples': 10.718212092181014,
 'nll': inf,
 'time': 33.64,
 'rmsle': 0.39996516029967116}

## 3.5 LSF

In [ ]:
lsf_params = {'min_bin_size': np.log(len(train_df))**2}

base_estimator = lightgbm_full_train_reg.model

start_time = time.perf_counter()
  
# fitting model on train set with early stopping on valid set
# since we cannot apply validation we are only training on the full train dataset once
lsf_reg = LSF(vectorizer_with_nan, target_transformer=target_transformer, base_model=base_estimator, model_trained=True, **lsf_params)
lsf_reg.fit(train_df, TARGET, verbose=True)

# predicting on test set with our fully trained model
lsf_pred = lsf_reg.predict(test_df, quantiles=[0.05, 0.1, 0.5, 0.9, 0.95], prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES, PredEnum.SAMPLES])
lsf_metrics = lsf_reg.metrics(np.array(test_df[TARGET]), lsf_pred, confidence_interval_quantiles=[0.1,0.9])
lsf_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], lsf_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
lsf_metrics['time'] = full_time

0d8779c737ee4799bc99f5f09c224ec0
Elapsed time for fitting LSF model: 12.69 s
/local_disk0/.ephemeral_nfs/envs/pythonEnv-bca11f19-7f4e-44cc-b4c3-13582dd2edf3/lib/python3.9/site-packages/lidl_x_tum_uncertainty_estimation/uncertainty_estimation_models.py:1628: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  samples = np.array([np.array(samples_for_one_input) for samples_for_one_input in samples])
path: models/model_lsf_w_lightgbm.pkl
dirname: models
filename: model_lsf_w_lightgbm.pkl
artifact_path: models
path: /tmp/tmp74aslc4y/model_lsf_w_lightgbm.pkl
tmp_path: /tmp/tmp74aslc4y/model_lsf_w_lightgbm.pkl
path: predictions/pointpredictions_lsf_w_lightgbm.csv
dirname: predictions
filename: pointpredictions_lsf_w_lightgbm.csv
artifact_path: predictions
path: /tmp/tmp0i0miqqn/p

In [ ]:
lsf_metrics

## 3.6 Conformalized Quantile Regression with MAPIE

In [ ]:
cqr_base_lgbm_params = {
    "boosting_type": 'gbdt',
    "objective": 'quantile',
    "n_jobs": -1, 
    "min_split_gain": 0.0,
    "min_data_in_leaf": 20,
    "max_bin": 255,
    "num_leaves": 31,
    "lambda": 0.0,
    "max_depth": -1,
    "learning_rate": 0.1,
    "feature_fraction": 1.0,
    "bagging_fraction": 1.0,
    "bagging_freq": 0,
    "seed": 1,
    "n_estimators": 2000, 
}

start_time = time.perf_counter()
    
cqr_best_iteration = int(np.mean([lightgbm_quant_reg.best_iterations[q] for q in [0.1, 0.5, 0.9]]))
print("Using best_iteration parameter from LGBMQuantileRegressor: ", cqr_best_iteration)
# fitting model on train+val set with best_iteration
cqr_full_fit_params = {**cqr_base_lgbm_params, "n_estimators": cqr_best_iteration}
cqr_full_train_reg = CQR(vectorizer_with_nan, target_transformer=target_transformer)
cqr_full_train_reg.fit(train_df, TARGET, train_cal_split=0.2, group_after=None, params=cqr_base_lgbm_params, alpha=0.2, verbose=True)


# predicting on test set with our fully trained model
cqr_pred = cqr_full_train_reg.predict(test_df)
cqr_metrics = cqr_full_train_reg.metrics(test_df[TARGET], cqr_pred)
cqr_metrics['rmsle'] = np.sqrt(mean_squared_log_error(test_df[TARGET], cqr_pred[PredEnum.POINT_ESTIMATES]))

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
cqr_metrics['time'] = full_time

e0ab48e7c59247598360766878de3b56
Using best_iteration parameter from LGBMQuantileRegressor:  84
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.0. Current value: lambda_l2=0.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
/local_disk0/.ephemeral_nfs/envs/pythonEnv-6596d653-83bd-47be-885f-51f7d7a2b7ff/lib/python3.9/site-packages/mapie/quantile_regression.py:196: UserWarning: WARNING: The alpha that is set needs to be the same as the alpha of your prefitted model in the following order [alpha/2, 1 - alpha/2, 0.5]
  warnings.warn(

In [ ]:
cqr_metrics